In [28]:
import db as db
import sqlalchemy as sql
from db import *
%run util.py

hostname = "cc1"
db.init(hostname)

'mysql://cc1:3306/project_b'

# coCLgenResults

In [ ]:
TABLE_NAME = "CLgen"
TABLE = coCLgenResult

In [21]:
import cldrive
import sys

def get_co_report(result_id: int, table=coCLgenResult, report_type: str="bf"):
    with Session(commit=False) as s:
        result = s.query(table).filter(table.id == result_id).first()

        if not result:
            raise KeyError(f"no result with ID {result_id}")

        flags = result.params.to_flags()
        program = result.program

        # generate bug report
        now = datetime.datetime.utcnow().isoformat()

        bug_type = {
            "bf": "compilation failure",
            "bc": "compiler crash",
            "c": "runtime crash",
        }[report_type]

        report_id = crypto.md5_str(table.__name__) + "-" + str(result.id)
        result_output = '\n'.join([f'//   {line}' for line in result.stderr.strip().split('\n')
                                   if not line.startswith("+ ")])
        header = f"""
// bug report {report_id}.c
// 
// Metadata:
//   OpenCL platform:        {result.testbed.platform}
//   OpenCL device:          {result.testbed.device}
//   Driver version:         {result.testbed.driver}
//   OpenCL version:         {result.testbed.opencl}
//   Host operating system:  {result.testbed.host}
//   OpenCL optimizations:   {result.params.optimizations_on_off}
//
// Output:
{result_output}
//   [Return code {result.status}]
//\
"""
        if isinstance(program, CLgenProgram):
            src = s.query(CLgenHarness).filter(
                CLgenHarness.program_id == program.id,
                CLgenHarness.params_id == result.params.id).first().src
        else:
            src = cldrive.emit_c(
                env=cldrive.make_env(),
                src=result.program.src,
                compile_only=True,
                create_kernel=result.params.build_kernel,
                gsize=None, lsize=None, inputs=None,
            )
        return (header + src).strip()

Segfaults:

In [5]:
from collections import defaultdict
from labm8 import fs

outbox = fs.path("outbox", "coCLgenResults", "segfaults")
fs.mkdir(outbox)
with Session(commit=True) as s:
    q = s.query(TABLE)\
        .filter(TABLE.status == 139,
                sql.or_(TABLE.submitted == None, TABLE.submitted == 0),
                sql.or_(TABLE.dupe == None, TABLE.dupe == 0))\
        .order_by(TABLE.id)

    dupes = 0
    errs = set()
    for result in q:
        key = result.testbed_id, result.program_id

        if key in errs:
            dupes += 1
            result.dupe = 1
            continue
        errs.add(key)

        vendor = PLATFORMS_2_VENDORS[result.testbed.platform]
        outpath = fs.path(outbox, f"bug-report-{vendor}-{result.id}.c")            

        report = get_co_report(**{
            "result_id": result.id,
            "table": coCLgenResult,
        })

        with open(outpath, "w") as outfile:
            print(report, file=outfile)
        result.submitted = 1
        print(outpath)
print("done")
print(f"{dupes} duplicates flagged")

outbox/coCLgenResults/segfaults/bug-report-intel-69539.c
done
0 duplicates flagged


# CLGenResults

In [26]:
TABLE_NAME = "CLgen"
TABLE = CLgenResult

In [29]:
outbox = fs.path("outbox", "CLgenResults", "bc")
fs.mkdir(outbox)
with Session(commit=True) as s:
    q = s.query(TABLE)\
        .filter(TABLE.outcome == "bc",
                sql.or_(TABLE.submitted == None, TABLE.submitted == 0),
                sql.or_(TABLE.dupe == None, TABLE.dupe == 0))\
        .order_by(TABLE.id)

    dupes = 0
    errs = set()
    for result in q:
        key = result.testbed_id, result.program_id

        if key in errs:
            dupes += 1
            result.dupe = 1
            continue
        errs.add(key)

        vendor = PLATFORMS_2_VENDORS[result.testbed.platform]
        outpath = fs.path(outbox, f"bug-report-{vendor}-{result.id}.c")            

        report = get_co_report(**{
            "result_id": result.id,
            "table": CLgenResult,
            "report_type": "bc",
        })

        with open(outpath, "w") as outfile:
            print(report, file=outfile)
        result.submitted = 1
        print(outpath)
print("done")
print(f"{dupes} duplicates flagged")

outbox/CLgenResults/bc/bug-report-oclgrind-503963.c
outbox/CLgenResults/bc/bug-report-intel-503984.c
outbox/CLgenResults/bc/bug-report-oclgrind-504013.c
outbox/CLgenResults/bc/bug-report-intel-504038.c
outbox/CLgenResults/bc/bug-report-oclgrind-504115.c
outbox/CLgenResults/bc/bug-report-intel-504127.c
outbox/CLgenResults/bc/bug-report-oclgrind-504240.c
outbox/CLgenResults/bc/bug-report-intel-504377.c
outbox/CLgenResults/bc/bug-report-oclgrind-504457.c
outbox/CLgenResults/bc/bug-report-oclgrind-504495.c
outbox/CLgenResults/bc/bug-report-oclgrind-504500.c
outbox/CLgenResults/bc/bug-report-oclgrind-504621.c
outbox/CLgenResults/bc/bug-report-oclgrind-504626.c
outbox/CLgenResults/bc/bug-report-pocl-504709.c
outbox/CLgenResults/bc/bug-report-pocl-504716.c
outbox/CLgenResults/bc/bug-report-oclgrind-504817.c
outbox/CLgenResults/bc/bug-report-pocl-504887.c
outbox/CLgenResults/bc/bug-report-intel-504900.c
outbox/CLgenResults/bc/bug-report-intel-504935.c
outbox/CLgenResults/bc/bug-report-pocl-505

outbox/CLgenResults/bc/bug-report-intel-509682.c
outbox/CLgenResults/bc/bug-report-pocl-509699.c
outbox/CLgenResults/bc/bug-report-intel-509701.c
outbox/CLgenResults/bc/bug-report-intel-509706.c
outbox/CLgenResults/bc/bug-report-intel-509745.c
outbox/CLgenResults/bc/bug-report-intel-509823.c
outbox/CLgenResults/bc/bug-report-intel-509871.c
outbox/CLgenResults/bc/bug-report-intel-509887.c
outbox/CLgenResults/bc/bug-report-intel-509961.c
outbox/CLgenResults/bc/bug-report-pocl-509983.c
outbox/CLgenResults/bc/bug-report-intel-509990.c
outbox/CLgenResults/bc/bug-report-intel-510080.c
outbox/CLgenResults/bc/bug-report-intel-510225.c
outbox/CLgenResults/bc/bug-report-pocl-510229.c
outbox/CLgenResults/bc/bug-report-intel-510257.c
outbox/CLgenResults/bc/bug-report-pocl-510367.c
outbox/CLgenResults/bc/bug-report-pocl-510418.c
outbox/CLgenResults/bc/bug-report-pocl-510530.c
outbox/CLgenResults/bc/bug-report-pocl-510598.c
outbox/CLgenResults/bc/bug-report-pocl-510686.c
outbox/CLgenResults/bc/bug-r

outbox/CLgenResults/bc/bug-report-intel-522943.c
outbox/CLgenResults/bc/bug-report-intel-522982.c
outbox/CLgenResults/bc/bug-report-intel-522989.c
outbox/CLgenResults/bc/bug-report-intel-523011.c
outbox/CLgenResults/bc/bug-report-intel-523299.c
outbox/CLgenResults/bc/bug-report-intel-523419.c
outbox/CLgenResults/bc/bug-report-intel-523468.c
outbox/CLgenResults/bc/bug-report-intel-523535.c
outbox/CLgenResults/bc/bug-report-pocl-523629.c
outbox/CLgenResults/bc/bug-report-intel-523715.c
outbox/CLgenResults/bc/bug-report-intel-523885.c
outbox/CLgenResults/bc/bug-report-pocl-523956.c
outbox/CLgenResults/bc/bug-report-intel-523979.c
outbox/CLgenResults/bc/bug-report-intel-524246.c
outbox/CLgenResults/bc/bug-report-pocl-524388.c
outbox/CLgenResults/bc/bug-report-intel-524414.c
outbox/CLgenResults/bc/bug-report-intel-524419.c
outbox/CLgenResults/bc/bug-report-intel-524563.c
outbox/CLgenResults/bc/bug-report-intel-524616.c
outbox/CLgenResults/bc/bug-report-intel-524638.c
outbox/CLgenResults/bc/

outbox/CLgenResults/bc/bug-report-pocl-535366.c
outbox/CLgenResults/bc/bug-report-pocl-535401.c
outbox/CLgenResults/bc/bug-report-pocl-535437.c
outbox/CLgenResults/bc/bug-report-intel-535573.c
outbox/CLgenResults/bc/bug-report-intel-535600.c
outbox/CLgenResults/bc/bug-report-intel-535711.c
outbox/CLgenResults/bc/bug-report-intel-535853.c
outbox/CLgenResults/bc/bug-report-intel-536033.c
outbox/CLgenResults/bc/bug-report-intel-536048.c
outbox/CLgenResults/bc/bug-report-intel-536174.c
outbox/CLgenResults/bc/bug-report-pocl-536176.c
outbox/CLgenResults/bc/bug-report-intel-536197.c
outbox/CLgenResults/bc/bug-report-intel-536199.c
outbox/CLgenResults/bc/bug-report-intel-536206.c
outbox/CLgenResults/bc/bug-report-intel-536225.c
outbox/CLgenResults/bc/bug-report-intel-536367.c
outbox/CLgenResults/bc/bug-report-intel-536423.c
outbox/CLgenResults/bc/bug-report-intel-536448.c
outbox/CLgenResults/bc/bug-report-intel-536501.c
outbox/CLgenResults/bc/bug-report-pocl-536601.c
outbox/CLgenResults/bc/bu

outbox/CLgenResults/bc/bug-report-intel-548430.c
outbox/CLgenResults/bc/bug-report-intel-548447.c
outbox/CLgenResults/bc/bug-report-intel-548551.c
outbox/CLgenResults/bc/bug-report-intel-548563.c
outbox/CLgenResults/bc/bug-report-intel-548729.c
outbox/CLgenResults/bc/bug-report-intel-548792.c
outbox/CLgenResults/bc/bug-report-intel-548872.c
outbox/CLgenResults/bc/bug-report-intel-548966.c
outbox/CLgenResults/bc/bug-report-intel-549038.c
outbox/CLgenResults/bc/bug-report-intel-549090.c
outbox/CLgenResults/bc/bug-report-intel-549151.c
outbox/CLgenResults/bc/bug-report-intel-549351.c
outbox/CLgenResults/bc/bug-report-pocl-549527.c
outbox/CLgenResults/bc/bug-report-intel-549722.c
outbox/CLgenResults/bc/bug-report-intel-549766.c
outbox/CLgenResults/bc/bug-report-pocl-549820.c
outbox/CLgenResults/bc/bug-report-intel-549928.c
outbox/CLgenResults/bc/bug-report-intel-549978.c
outbox/CLgenResults/bc/bug-report-intel-549997.c
outbox/CLgenResults/bc/bug-report-pocl-550005.c
outbox/CLgenResults/bc/

outbox/CLgenResults/bc/bug-report-intel-574115.c
outbox/CLgenResults/bc/bug-report-intel-574120.c
outbox/CLgenResults/bc/bug-report-intel-574242.c
outbox/CLgenResults/bc/bug-report-pocl-574335.c
outbox/CLgenResults/bc/bug-report-intel-574385.c
outbox/CLgenResults/bc/bug-report-intel-574398.c
outbox/CLgenResults/bc/bug-report-intel-574505.c
outbox/CLgenResults/bc/bug-report-intel-574589.c
outbox/CLgenResults/bc/bug-report-intel-574624.c
outbox/CLgenResults/bc/bug-report-intel-574636.c
outbox/CLgenResults/bc/bug-report-intel-574644.c
outbox/CLgenResults/bc/bug-report-intel-574645.c
outbox/CLgenResults/bc/bug-report-intel-574656.c
outbox/CLgenResults/bc/bug-report-intel-574667.c
outbox/CLgenResults/bc/bug-report-intel-574698.c
outbox/CLgenResults/bc/bug-report-intel-574838.c
outbox/CLgenResults/bc/bug-report-intel-574839.c
outbox/CLgenResults/bc/bug-report-intel-574854.c
outbox/CLgenResults/bc/bug-report-pocl-574923.c
outbox/CLgenResults/bc/bug-report-intel-575054.c
outbox/CLgenResults/bc

outbox/CLgenResults/bc/bug-report-intel-583290.c
outbox/CLgenResults/bc/bug-report-intel-583338.c
outbox/CLgenResults/bc/bug-report-intel-583358.c
outbox/CLgenResults/bc/bug-report-intel-583410.c
outbox/CLgenResults/bc/bug-report-intel-583421.c
outbox/CLgenResults/bc/bug-report-intel-583455.c
outbox/CLgenResults/bc/bug-report-intel-583509.c
outbox/CLgenResults/bc/bug-report-intel-583613.c
outbox/CLgenResults/bc/bug-report-intel-583704.c
outbox/CLgenResults/bc/bug-report-intel-583762.c
outbox/CLgenResults/bc/bug-report-intel-583789.c
outbox/CLgenResults/bc/bug-report-intel-583808.c
outbox/CLgenResults/bc/bug-report-intel-583870.c
outbox/CLgenResults/bc/bug-report-intel-583882.c
outbox/CLgenResults/bc/bug-report-intel-583910.c
outbox/CLgenResults/bc/bug-report-intel-583929.c
outbox/CLgenResults/bc/bug-report-intel-583948.c
outbox/CLgenResults/bc/bug-report-intel-583955.c
outbox/CLgenResults/bc/bug-report-intel-583975.c
outbox/CLgenResults/bc/bug-report-intel-584049.c
outbox/CLgenResults/

outbox/CLgenResults/bc/bug-report-intel-591434.c
outbox/CLgenResults/bc/bug-report-intel-591478.c
outbox/CLgenResults/bc/bug-report-intel-591608.c
outbox/CLgenResults/bc/bug-report-intel-591619.c
outbox/CLgenResults/bc/bug-report-intel-591669.c
outbox/CLgenResults/bc/bug-report-intel-591693.c
outbox/CLgenResults/bc/bug-report-intel-591701.c
outbox/CLgenResults/bc/bug-report-intel-591709.c
outbox/CLgenResults/bc/bug-report-pocl-591813.c
outbox/CLgenResults/bc/bug-report-intel-591847.c
outbox/CLgenResults/bc/bug-report-intel-591884.c
outbox/CLgenResults/bc/bug-report-intel-591982.c
outbox/CLgenResults/bc/bug-report-pocl-591993.c
outbox/CLgenResults/bc/bug-report-pocl-592034.c
outbox/CLgenResults/bc/bug-report-intel-592046.c
outbox/CLgenResults/bc/bug-report-intel-592182.c
outbox/CLgenResults/bc/bug-report-pocl-592235.c
outbox/CLgenResults/bc/bug-report-intel-592314.c
outbox/CLgenResults/bc/bug-report-pocl-592330.c
outbox/CLgenResults/bc/bug-report-pocl-592342.c
outbox/CLgenResults/bc/bug